# **Dealing with 100+ classes, especially when some are visually similar (like different flower species), requires a more powerful technique than a simple CNN built from scratch. We'll use the Oxford Flowers 102 dataset and implement Transfer Learning using a pre-trained model like MobileNetV2.**



In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#  Hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 102 # The Oxford Flowers 102 dataset has 102 classes
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
# Load the Dataset
# Oxford Flowers 102 has 'train', 'validation', and 'test' splits
(train_ds_raw, val_ds_raw, test_ds_raw), metadata = tfds.load(
    'oxford_flowers102',
    split=['train', 'validation', 'test'],
    as_supervised=True,
    with_info=True,
)

print(f"Number of training examples: {metadata.splits['train'].num_examples}")
print(f"Number of classes: {metadata.features['label'].num_classes}")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.5W7C5J_2.1.1/oxford_flowers102-test.tfrecord*…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.5W7C5J_2.1.1/oxford_flowers102-validation.tfr…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.
Number of training examples: 1020
Number of classes: 102


In [5]:
#Preprocessing Function
def preprocess(image, label):
    # Resize the image to the required input size for MobileNetV2
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    # Normalize images from [0, 255] to [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


In [ ]:
# Augmentation Layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])